In [ ]:
# A proof of concept showing using a genetic algorithm with our environment.
# It is similar to https://github.com/DEAP/deap/blob/a0b78956e28387785e3bb6e2b4b1f1b32c2b3883/examples/ga/onemax_short.py

import array
import random

import numpy as np

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
import gym
from environments.SimpleEnv import SimpleEnv

from ray.rllib.agents import ppo
from ray import tune
from ray.rllib.policy.policy import Policy
from ray.rllib.env.multi_agent_env import MultiAgentEnv
import ray

ray.init()


creator.create('FitnessMax', base.Fitness, weights=(1.0, ))
creator.create('Individual', array.array, typecode='d',
               fitness=creator.FitnessMax)

toolbox = base.Toolbox()

toolbox.register('attr', random.uniform, -1, 1)
toolbox.register('individual', tools.initRepeat, creator.Individual,
                 toolbox.attr, 2)
toolbox.register('population', tools.initRepeat, list,
                 toolbox.individual)

n_agents = 3

def evaluate(pop):
    """Runs the environment. Selects random agents from pop.
  It returns the total true reward as the fitness.
  TODO: Assumes n_agents is 2.
  TODO: This is very slow, can we train the whole population at the same time?
  """
    
    #Select random individuals from pop and create the reward weights
    pop = np.array(pop)
    reward_weights = pop
    #print(individual, reward_weights)
    
    #env is only to get action space and observation space
    env = SimpleEnv(config={
        'n_agents': n_agents,
        'n_vars': 2,
        'reward_weights': reward_weights,
        'max_step_count': 10,
    })
    
    #Callback functions to keep track of true reward while training
    def on_episode_start(info):
        episode = info["episode"]
        episode.user_data["true_rewards"] = np.zeros(n_agents)#TODO: change to n_agents

    def on_episode_step(info):
        episode = info["episode"]
        env = info["env"]
        #print(env.env_states[0].env.last_true_reward)
        true_reward = env.env_states[0].env.last_true_reward
        episode.user_data["true_rewards"] += true_reward

    def on_episode_end(info):
        episode = info["episode"]
        true_reward = episode.user_data["true_rewards"]
        for i, r in enumerate(true_reward):
            episode.custom_metrics["true_reward_agent_" + str(i)] = r
    config={
        "multiagent": {
            "policies": {
            },
            "policy_mapping_fn":
                lambda agent_id:
                    agent_id
        },
        'env_config': {
            'n_agents': n_agents,
            'n_vars': 2,
            'reward_weights': reward_weights,
            'max_step_count': 10,
        },
        "callbacks": {
            "on_episode_start": on_episode_start,
            "on_episode_step": on_episode_step,
            "on_episode_end": on_episode_end,
        },
    }
    for i in range(n_agents):
        config['multiagent']['policies']['agent_' + str(i)] = (None, env.observation_space, env.action_space, {})
    trainer = ppo.PPOTrainer(env=SimpleEnv, config=config)
    
    true_reward_mean = []
    for i in range(1):
        print('TRAINING', i)
        custom_metrics = trainer.train()['custom_metrics']  # distributed training step
        print(custom_metrics)
        for i in range(n_agents):
            true_reward_mean.append((custom_metrics['true_reward_agent_' + str(i) + '_mean'], ))
        
    #print('true reward', trainer.collect_metrics()['custom_metrics']['true_reward_mean'])
    return true_reward_mean


toolbox.register('evaluate', evaluate)
toolbox.register('mate', tools.cxTwoPoint)
toolbox.register('mutate', tools.mutFlipBit, indpb=0.05)
toolbox.register('select', tools.selTournament, tournsize=3)

def evolve(population, toolbox, cxpb, mutpb, ngen, stats=None, 
           halloffame=None, verbose=__debug__):
    """
    Almost identical to deap.algorithms.eaSimple. 
    
    TODO: Sometimes all individuals become identical for some reason
    Runtime is O(pop size * ngen * RL iterations) with a huge constant. 
    Easily parralizable, but we can remove the pop size factor by training the
    entire population at once. Requires environments to support many agents.
    """
    
    logbook = tools.Logbook()
    logbook.header = ['gen', 'nevals'] + (stats.fields if stats else [])

    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in population]
    #print('population1', population)
    fitnesses = evaluate(invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    #print('population2', population)
    if halloffame is not None:
        halloffame.update(population)

    record = stats.compile(population) if stats else {}
    logbook.record(gen=0, nevals=len(invalid_ind), **record)
    if verbose:
        print(logbook.stream)

    # Begin the generational process
    for gen in range(1, ngen + 1):
        # Select the next generation individuals
        print('population', population)
        offspring = toolbox.select(population, len(population))

        # Vary the pool of individuals
        offspring = algorithms.varAnd(offspring, toolbox, cxpb, mutpb)

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring]
        fitnesses = evaluate(invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        # Update the hall of fame with the generated individuals
        if halloffame is not None:
            halloffame.update(offspring)

        # Replace the current population by the offspring
        population[:] = offspring

        # Append the current generation statistics to the logbook
        record = stats.compile(population) if stats else {}
        logbook.record(gen=gen, nevals=len(invalid_ind), **record)
        if verbose:
            print(logbook.stream)

    return population, logbook

def main():
    pop = toolbox.population(n=n_agents)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register('avg', np.mean)
    stats.register('std', np.std)
    stats.register('min', np.min)
    stats.register('max', np.max)

    (pop, log) = evolve(
        pop,
        toolbox,
        cxpb=0.5,
        mutpb=0.2,
        ngen=3,
        stats=stats,
        halloffame=hof,
        verbose=True,
        )

    print ('pop', pop)

    return (pop, log, hof)


if __name__ == '__main__':
    main()


/home/victor/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
2020-05-01 21:20:50,248	INFO resource_spec.py:212 -- Starting Ray with 2.0 GiB memory available for workers and up to 1.01 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-05-01 21:20:50,705	INFO services.py:1170 -- View the Ray dashboard at localhost:8266
2020-05-01 21:20:50,875	INFO trainer.py:421 -- Tip: set 'eager': true or the --eager flag to enable TensorFlow eager execution
2020-05-01 21:20:50,947	ERROR syncer.py:39 -- Log sync requires rsync to be installed.
2020-05-01 21:20:50,961	WARNING deprecation.py:30 -- DeprecationWarning: `callbacks dict interface` has been deprecated. Use `a class extending rllib.

(pid=27296) /home/victor/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
(pid=27296)   from ._conv import register_converters as _register_converters
(pid=27297) /home/victor/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
(pid=27297)   from ._conv import register_converters as _register_converters
(pid=27296) 2020-05-01 21:20:58,813	WARNING deprecation.py:30 -- DeprecationWarning: `callbacks dict interface` has been deprecated. Use `a class extending rllib.agents.callbacks.DefaultCallbacks` instead. This will raise an error in the future!
(pid=27297) 2020-05-01 21:20:58,799	WARNING deprecation.py:30 -- Deprecati

2020-05-01 21:21:10,664	INFO trainable.py:180 -- _setup took 19.716 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2020-05-01 21:21:10,665	INFO trainable.py:217 -- Getting current IP.
2020-05-01 21:21:10,667	WARNING util.py:37 -- Install gputil for GPU system monitoring.


TRAINING 0


2020-05-01 21:21:31,699	ERROR syncer.py:39 -- Log sync requires rsync to be installed.
2020-05-01 21:21:31,703	WARNING deprecation.py:30 -- DeprecationWarning: `callbacks dict interface` has been deprecated. Use `a class extending rllib.agents.callbacks.DefaultCallbacks` instead. This will raise an error in the future!
2020-05-01 21:21:31,707	WARNING deprecation.py:30 -- DeprecationWarning: `callbacks dict interface` has been deprecated. Use `a class extending rllib.agents.callbacks.DefaultCallbacks` instead. This will raise an error in the future!


{'true_reward_agent_0_mean': -0.21108026425921708, 'true_reward_agent_0_min': -14.763866104185581, 'true_reward_agent_0_max': 19.05920815654099, 'true_reward_agent_1_mean': 0.34137869184080044, 'true_reward_agent_1_min': -10.993358571082354, 'true_reward_agent_1_max': 14.195867132395506, 'true_reward_agent_2_mean': 0.1775818211529986, 'true_reward_agent_2_min': -14.979214802384377, 'true_reward_agent_2_max': 16.57559995725751}
gen	nevals	avg     	std     	min     	max     
0  	3     	0.102627	0.231684	-0.21108	0.341379
population [array('d', [0.3642682466388645, 0.07403647421335169]), array('d', [-0.42947817127639665, 0.48133058781025717]), array('d', [-0.7725784024785114, 0.394858947624261])]


In [ ]:
#I had problems with rllib running out of memory after running many tests. Installing the ray nightly build fixed it
ray.shutdown()